<a href="https://colab.research.google.com/github/NUELBUNDI/Machine-Learning-Projects/blob/main/ML_Logistic_Regression_Employee_Attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
url="https://raw.githubusercontent.com/NUELBUNDI/Machine-Learning-Projects/main/Employee-Attrition.csv"

df=pd.read_csv(url)
print(f'Rows x Columns is :{df.shape}')

In [ ]:
print(f' The Column Names are :{df.columns}')

In [ ]:
# first five Columns
df.head(2)

In [ ]:
df.info()

In [ ]:
df.columns.tolist()

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

## Exploratory Data Analysis and Visualization


In [ ]:
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['figure.figsize']=(10,6)
matplotlib.rcParams['figure.facecolor']='#00000000'

In [ ]:
def HistogramDistributionInData(name, color):
  fig= px.histogram(df,
                    x=name,
                    marginal='box',
                    title=f'Distribution of {name}',
                    color=color)
  fig.update_layout(bargap=0.1)
  
  return fig.show()


In [ ]:
HistogramDistributionInData('Age','Attrition')

The Age is Normal distribution
The staff between the age of 20-35 have high no's of attrition

In [ ]:
HistogramDistributionInData('PerformanceRating','Attrition')

High No's of attrition for staff under perfomance rating of 3 compared to performance rating of 4


In [ ]:
df.YearsInCurrentRole.unique()

In [ ]:
HistogramDistributionInData('YearsInCurrentRole','Attrition')

In [ ]:
px.bar(y=df.YearsInCurrentRole.value_counts(),
       title='Count by the No of Years in the Current Role',
       )


In [ ]:
HistogramDistributionInData('MonthlyIncome','Attrition')

In [ ]:
df.columns

In [ ]:
df.drop(columns=['EmployeeNumber'],inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_val_df , test_df =train_test_split(df,test_size=0.2, random_state=42)

train_df , val_df =train_test_split(train_val_df,random_state=42)



print(f'df :{df.shape}')

print(f'train_df :{train_df.shape}')

print(f'test_df :{test_df.shape}')

print(f'val_df :{val_df.shape}')

In [ ]:
input_cols=[]
for col in df.columns:
  if col != 'Attrition':
    input_cols.append(col)

target_col= 'Attrition'

In [ ]:
print(input_cols)

print(target_col)

In [ ]:
train_inputs= train_df[input_cols].copy()
train_targets= train_df[target_col]


val_inputs=val_df[input_cols].copy()
val_targets= val_df[target_col].copy()

test_inputs= test_df[input_cols].copy()
test_targets= test_df[target_col].copy()

In [ ]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols=train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
train_inputs[numeric_cols].describe()

# Scaling Numerical Features



In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler= MinMaxScaler()

In [ ]:
scaler.fit(df[numeric_cols])

In [ ]:
print(f'Minimum : {list(scaler.data_min_)}')

print(f'Max : {list(scaler.data_max_)}')

In [ ]:
train_inputs[numeric_cols]=scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [ ]:
df.nunique()

In [ ]:
df[numeric_cols].nunique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder= OneHotEncoder(sparse=False,handle_unknown='ignore')

In [ ]:
encoder.fit(df[categorical_cols])

In [ ]:
encoder.categories_

In [ ]:
encoded_cols= list(encoder.get_feature_names_out(categorical_cols))
print(encoded_cols)
print(len(encoded_cols))

In [ ]:
train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
test_inputs

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model=LogisticRegression(solver='liblinear')

In [ ]:
model.fit(train_inputs[numeric_cols + encoded_cols],train_targets)

In [ ]:
print(numeric_cols + encoded_cols)

In [ ]:
print(model.coef_.tolist())

In [ ]:
print(model.intercept_)

### Making Predictions and Evaluating the Model

In [ ]:
X_train = train_inputs[numeric_cols + encoded_cols]
X_val = val_inputs[numeric_cols + encoded_cols]
X_test = test_inputs[numeric_cols + encoded_cols]

In [ ]:
train_preds= model.predict(X_train)

train_probs=model.predict_proba(X_train)
train_probs

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(train_targets,train_preds)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(train_targets, train_preds, normalize='true')

In [ ]:
def predict_and_plot(inputs, targets, name=''):
  preds=model.predict(inputs)

  accuracy=accuracy_score(targets, preds)
  print(f'Accuracy : {accuracy*100:.2f}')

  cf =confusion_matrix(targets,preds,normalize ='true')
  plt.figure()
  sns.heatmap(cf,annot=True)
  plt.xlabel('Prediction')
  plt.ylabel('Target')
  plt.title((f'{name} Confusion Matrix'))

  return preds

In [ ]:
train_preds = predict_and_plot(X_train, train_targets, 'Training')

In [ ]:
val_preds = predict_and_plot(X_val, val_targets, 'Validatiaon')

In [ ]:
test_preds = predict_and_plot(X_test, test_targets, 'Test')

In [ ]:
def random_guess(inputs):
    return np.random.choice(["No", "Yes"], len(inputs))

def all_no(inputs):
  return np.full(len(inputs), "No")


In [ ]:
accuracy_score(test_targets, random_guess(X_test))

In [ ]:
accuracy_score(test_targets, all_no(X_test))

In [ ]:
def predict_input(single_input):
    input_df = pd.DataFrame([single_input])
    input_df[numeric_cols] = scaler.transform(input_df[numeric_cols])
    input_df[encoded_cols] = encoder.transform(input_df[categorical_cols])
    X_input = input_df[numeric_cols + encoded_cols]
    pred = model.predict(X_input)[0]
    prob = model.predict_proba(X_input)[0][list(model.classes_).index(pred)]
    return pred, prob

In [ ]:
# df.head(1).to_dict('records')

In [ ]:
new_input= {
    'Age': 20,
  'Attrition': 'Yes',
  'BusinessTravel': 'Travel_Rarely',
  'DailyRate': 102,
  'Department': 'Sales',
  'DistanceFromHome': 2,
  'Education': 2,
  'EducationField': 'Life Sciences',
  'EmployeeCount': 1,
  'EnvironmentSatisfaction': 2,
  'Gender': 'Female',
  'HourlyRate': 94,
  'JobInvolvement': 3,
  'JobLevel': 2,
  'JobRole': 'Sales Executive',
  'JobSatisfaction': 4,
  'MaritalStatus': 'Single',
  'MonthlyIncome': 100000,
  'MonthlyRate': 19479,
  'NumCompaniesWorked': 8,
  'Over18': 'Y',
  'OverTime': 'Yes',
  'PercentSalaryHike': 11,
  'PerformanceRating': 3,
  'RelationshipSatisfaction': 1,
  'StandardHours': 100,
  'StockOptionLevel': 0,
  'TotalWorkingYears': 15,
  'TrainingTimesLastYear': 2,
  'WorkLifeBalance': 1,
  'YearsAtCompany': 5,
  'YearsInCurrentRole': 4,
  'YearsSinceLastPromotion': 5,
  'YearsWithCurrManager': 5}


In [ ]:

predict_input(new_input)